Import libraries

In [4]:
pip install pytorch-crf

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.9 MB/s eta 0:00:00a 0:00:01m
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
import pandas as pd
import os
import string
from unidecode import unidecode
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import AdamW, BertModel, AutoConfig, AutoTokenizer
from torch.optim import Adam
import torch.nn as nn
from transformers import AutoTokenizer, get_linear_schedule_with_warmup
import time;
import datetime
from sklearn.model_selection import train_test_split
from torchcrf import CRF

In [202]:
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

We will use the GPU: Tesla T4


Read data

In [6]:
nlp_train=pd.read_csv("/kaggle/input/nlp-dataset/train.csv", index_col=[0])
nlp_test=pd.read_csv("/kaggle/input/nlp-dataset/test.csv",index_col=[0],encoding="windows-1252") 

#nlp_train=pd.read_csv("/kaggle/input/nlp-project-train/train.csv", index_col=[0])
#nlp_test=pd.read_csv("/kaggle/input/nlp-project-train/test.csv",index_col=[0],encoding="windows-1252")

Functions to manipulate data

In [7]:
def convert_to_ascii(text):
    text = unidecode(text)
    text = text.replace("I", "ı").lower()
    text = text.replace("i", "ı")
    return text

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
        text = text.replace('  ', ' ')
    return text.strip()


In [8]:
def pad_sequence(sequence, max_length, padding_value=0):
    sequence = (sequence + [padding_value] * (max_length - len(sequence)))
    return sequence

def pad_sentences(encoded_sentences, max_sentence_length, max_word_length):
    padded_sentences = []
    for sentence in encoded_sentences:
        padded_words = [pad_sequence(word, max_word_length) for word in sentence]
        while len(padded_words) < max_sentence_length:
            padded_words.append([0] * max_word_length)
        padded_sentences.append(padded_words)
    return padded_sentences

In [9]:
def encode_character(char):
    diacritic_chars = "iöü"
    return 2 if char in diacritic_chars else 1

def encode_word(word):
    return [encode_character(char) for char in word]

def encode_sentences(text, max_word_length, max_sentence_length):
    sentences = text
    encoded_sentences = [[encode_word(word) for word in sentence.split()] for sentence in sentences]
    padded_encoded_sentences = pad_sentences(encoded_sentences, max_sentence_length, max_word_length)
    return padded_encoded_sentences

# Example usage
#text = ["Bu öykü", "Bu, Mustafa'nın hikayesi"]
#encoded_output = encode_sentences(text)
#print(encoded_output[0])

In [10]:
##
#  0 for letters which always be used with base format (we will encode as index)
#  1 for letters which is used as non-diacritic
#  2 for letters whic is used as diacritic
#(we will encode as index)

## example for word öykü -->  4x3 matrix,
# [0 0 1]
# [1 0 0]
# [1 0 0]
# [0 0 1]

## example for word mustafa
# [1 0 0]
# [0 1 0]
# [1 0 0]
# [1 0 0]
# [1 0 0]
# [1 0 0]
# [1 0 0]

In [11]:
nlp_train['Sentence'] = nlp_train['Sentence'].apply(remove_punctuations)
nlp_train["Label"] = nlp_train["Sentence"]
nlp_train["Sentence"] = nlp_train["Sentence"].apply(convert_to_ascii)

In [12]:
nlp_train["Label"][0:2]

ID
0    sınıf havuz ve açık deniz çalışmalarıyla tüm d...
1    bu standart sualtında kendini rahat hisseden h...
Name: Label, dtype: object

In [13]:
sentences_train, labels_train = nlp_train.Sentence.values, nlp_train.Label.values

Tokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")

tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

Functions to segment data

In [15]:
def segment_text(sentence, label, max_length=128, max_length_word=70, overlap=50):
    tokens = tokenizer.tokenize(sentence)

    # Check if the length of any word exceeds the maximum length allowed for a word
    if any(len(word) > max_length_word for word in sentence.split()):
        return None, None

    # Check if the total number of tokens exceeds the maximum length
    if len(tokens) > max_length:
        return None, None

    # If all checks pass, return the original sentence and label
    return sentence, label
    
    
def data_segments(sentences, labels, max_length=32):
    all_text = []
    all_labels = []

    for sentence, label in zip(sentences, labels):
        segment_s, segment_l = segment_text(sentence, label, max_length=max_length, overlap=50)
        if segment_s:
            all_text.append(segment_s)
            all_labels.append(segment_l)
            
    return all_text, all_labels

In [16]:
train_sentences_segment, train_labels_segment = data_segments(sentences_train, labels_train)

Token indices sequence length is longer than the specified maximum sequence length for this model (7014 > 512). Running this sequence through the model will result in indexing errors


In [17]:
tokenizer.tokenize(train_sentences_segment[25748])

['sıf', '##ren', '##ız']

In [18]:
labels_train_encoded = encode_sentences(train_labels_segment, max_word_length=70, max_sentence_length=40)

In [19]:
np.array(labels_train_encoded).shape

(37002, 40, 70)

In [20]:
indices=tokenizer.batch_encode_plus(train_sentences_segment,max_length=32,add_special_tokens=True, return_attention_mask=True,padding=True,truncation=True)
input_ids=indices["input_ids"]
attention_masks=indices["attention_mask"]
print(input_ids[0])
print(train_sentences_segment[0])
print(attention_masks[0])

[2, 3825, 8725, 1992, 24045, 2350, 2028, 15007, 2116, 3970, 2218, 13526, 1027, 26905, 24419, 3575, 1028, 1991, 21070, 2187, 16090, 5538, 14330, 2033, 2002, 9474, 2293, 3, 0, 0, 0, 0]
sınıf havuz ve acık denız calısmalarıyla tum dunyada gecerlı basarılı bır standart olusturmustur
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


In [21]:
def align_labels_with_tokens(labels_list, tokenized_input):
    aligned_labels = []
    for labels, input_ids in zip(labels_list, tokenized_input):
        word_ids = tokenizer.convert_ids_to_tokens(input_ids)  # Get the token strings
        aligned_label = []
        current_word = 0
        for word_piece, label in zip(word_ids, labels):
            if tokenizer.convert_tokens_to_ids(word_piece) == tokenizer.cls_token_id or \
               tokenizer.convert_tokens_to_ids(word_piece) == tokenizer.sep_token_id:
                # Special tokens have a label "ignore" or some specific label
                aligned_label.append([-100] * 70)  # Typically ignored in loss computation
            else:
                if word_piece.startswith("##"):
                    # Continuation of the previous word
                    aligned_label.append(labels[current_word])
                else:
                    # A new word
                    current_word += 1
                    aligned_label.append(labels[current_word])
        aligned_labels.append(aligned_label)
    return aligned_labels

# Adjust the `train_labels` structure to fit the exact format needed
aligned_train_labels = align_labels_with_tokens(labels_train_encoded, input_ids)


In [26]:
np.array(input_ids).shape

(37002, 32)

In [25]:
np.array(aligned_train_labels).shape

(37002, 32, 70)

In [228]:
"""
indices=tokenizer.batch_encode_plus(train_labels_segment,max_length=32,add_special_tokens=True, return_attention_mask=True,padding=True,truncation=True)
output_ids=indices["input_ids"]
print(output_ids[0])
print(train_labels_segment[0])
"""

[2, 3825, 8725, 1992, 2416, 4456, 24513, 2525, 5292, 5953, 4165, 1996, 5538, 27202, 2293, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sınıf havuz ve açık deniz çalışmalarıyla tüm dünyada geçerli başarılı bir standart oluşturmuştur


Prepare train and test data

In [28]:
# Use 99% for training and 1% for validation.
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,aligned_train_labels, 
                                                            random_state=42, test_size=0.2)
# Do the same for the masks.
train_masks, validation_masks, _, _ = train_test_split(attention_masks, aligned_train_labels,
                                             random_state=42, test_size=0.2)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels, dtype=torch.long)
validation_labels = torch.tensor(validation_labels, dtype=torch.long)
train_masks = torch.tensor(train_masks, dtype=torch.long)
validation_masks = torch.tensor(validation_masks, dtype=torch.long)


In [52]:
def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    #print("preds", preds.shape)
    #print("labels", labels.shape)
    pred_flat = np.argmax(preds, axis=2).flatten()
    print("argmax", pred_flat[0])
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

## Combined Model

In [53]:
# unique_labels = nlp_train['Label'].unique()
# num_labels = len(unique_labels)

# tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
# encoded_batch = tokenizer(train_sentences_segment, padding=True, truncation=True, return_tensors="pt")
# input_ids = encoded_batch['input_ids']
# attention_mask_train = encoded_batch['attention_mask']

# train_data = TensorDataset(train_inputs, train_masks, train_labels)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

In [48]:
# class BertBiLSTMCRF(nn.Module):
#     def __init__(self, bert_model, num_labels, lstm_hidden_dim):
#         super(BertBiLSTMCRF, self).__init__()
#         self.bert = BertModel.from_pretrained(bert_model)
#         self.lstm = nn.LSTM(input_size=self.bert.config.hidden_size,
#                             hidden_size=lstm_hidden_dim,
#                             num_layers=1,
#                             bidirectional=True,
#                             batch_first=True)
#         self.dropout = nn.Dropout(0.1)
#         self.classifier = nn.Linear(lstm_hidden_dim * 2, num_labels)
#         self.crf = CRF(num_labels, batch_first=True)

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.bert(input_ids, attention_mask=attention_mask)
#         sequence_output = outputs[0]
#         lstm_output, _ = self.lstm(sequence_output)
#         lstm_output = self.dropout(lstm_output)
#         logits = self.classifier(lstm_output)
#         if labels is not None:
#             loss = -self.crf(logits, labels, mask=attention_mask.byte())
#             return loss, logits
#         else:
#             return logits

#     def predict(self, input_ids, attention_mask=None):
#         with torch.no_grad():
#             logits = self.forward(input_ids, attention_mask)
#             return self.crf.decode(logits, mask=attention_mask.byte())
        
"""        
class BertBiLSTMCRF(nn.Module):
    def __init__(self, bert_model, num_labels, lstm_hidden_dim, lstm_layers=1):
        super(BertBiLSTMCRF, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.lstm = nn.LSTM(input_size=self.bert.config.hidden_size,
                            hidden_size=lstm_hidden_dim,
                            num_layers=lstm_layers,
                            bidirectional=True,
                            batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(lstm_hidden_dim * 2, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        lstm_output, _ = self.lstm(sequence_output)
        lstm_output = self.dropout(lstm_output)
        logits = self.classifier(sequence_output)
        if labels is not None:
            loss = -self.crf(logits, labels, mask=attention_mask.byte())  # Calculate loss using CRF
            return loss, logits
        else:
            return logits

    def predict(self, input_ids, attention_mask=None):
        with torch.no_grad():
            logits = self.forward(input_ids, attention_mask)
            return self.crf.decode(logits, mask=attention_mask.byte())
            
"""
"""
class BertCRF(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BertCRF, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        if labels is not None:
            loss = -self.crf(logits, labels, mask=attention_mask.byte())  # Calculate loss using CRF
            return loss, logits
        else:
            return logits

    def predict(self, input_ids, attention_mask=None):
        with torch.no_grad():
            logits = self.forward(input_ids, attention_mask)
            return self.crf.decode(logits, mask=attention_mask.byte())
"""
"""
class BertCRF(nn.Module):
    def __init__(self, bert_model, num_labels):
        super(BertCRF, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        loss = 0.0
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self.classifier.out_features)[active_loss]
                active_labels = labels.view(-1)[active_loss]
                loss = loss_fct(active_logits, active_labels)
            else:
                loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))
            return loss, logits
        else:
            return loss, logits

    def predict(self, input_ids, attention_mask=None):
        logits = self.forward(input_ids, attention_mask)
        return torch.argmax(logits, dim=-1)
"""

import torch.nn.functional as F

class DiacritizationModel(nn.Module):
    def __init__(self, bert_model, hidden_dim, num_words=32, num_chars=70, num_classes=3):
        super(DiacritizationModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model)
        self.gru = nn.GRU(self.bert.config.hidden_size, hidden_dim, batch_first=True)
        # Assuming each token output from GRU should decide the labels of multiple characters
        self.classifier = nn.Linear(hidden_dim, num_chars * num_classes)
        self.num_words = num_words
        self.num_chars = num_chars
        self.num_classes = num_classes

    def forward(self, input_ids, attention_mask):
        # BERT outputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # (batch_size, seq_length, hidden_size)

        # GRU processing
        gru_output, _ = self.gru(sequence_output)  # (batch_size, seq_length, hidden_dim)

        # Classification to character-level diacritic status
        prediction = self.classifier(gru_output)  # (batch_size, seq_length, num_chars * num_classes)
        # Reshape to make each position in the sequence have a (num_chars, num_classes) prediction
        prediction = prediction.view(-1, self.num_words, self.num_chars, self.num_classes)  # Reshaping to (batch_size, 32, 70, 3)

        return prediction


In [49]:
"""
def train_model(model, train_dataloader, validation_dataloader, device, epochs=4, gradient_accumulation_steps=1):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)
    criterion = nn.CrossEntropyLoss() 
    
    for epoch_i in range(epochs):
        print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
        total_train_loss = 0
        
        model.train()
        for step, batch in enumerate(train_dataloader):
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            
            model.zero_grad()        
            loss, logits = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            #loss = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            
            loss = loss / gradient_accumulation_steps
            loss.backward()
            
            total_train_loss += loss.item()
            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
            
            if step % 40 == 0 and not step == 0:
                print(f'  Batch {step:>5,} of {len(train_dataloader):>5,}. Loss: {loss.item():.2f}')
        
        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"  Average training loss: {avg_train_loss:.2f}")

        print("\nRunning Validation...")
        total_eval_accuracy = 0
        total_eval_loss = 0
        model.eval()

        for batch in validation_dataloader:
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
            
            with torch.no_grad():
                loss, logits = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
                #loss = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            total_eval_loss += loss.item()
            
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            #print("logits", logits.shape)
            #print("labels,", label_ids.shape)
            total_eval_accuracy += flat_accuracy(logits, label_ids)
        
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print(f"  Validation accuracy: {avg_val_accuracy:.2f}")
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        print(f"  Validation Loss: {avg_val_loss:.2f}")

    print("done")
"""

'\ndef train_model(model, train_dataloader, validation_dataloader, device, epochs=4, gradient_accumulation_steps=1):\n    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)\n    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)\n    criterion = nn.CrossEntropyLoss() \n    \n    for epoch_i in range(epochs):\n        print(f\'======== Epoch {epoch_i + 1} / {epochs} ========\')\n        total_train_loss = 0\n        \n        model.train()\n        for step, batch in enumerate(train_dataloader):\n            batch = tuple(t.to(device) for t in batch)\n            b_input_ids, b_input_mask, b_labels = batch\n            \n            model.zero_grad()        \n            loss, logits = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)\n            #loss = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)\n            \n            loss = loss / gradient_accumulation_steps\n            loss.backward()\n            \n  

In [50]:
import torch
import torch.nn as nn

def train_model(model, train_dataloader, validation_dataloader, device, epochs=4, gradient_accumulation_steps=1):
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.95)
    criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignoring padding index for loss calculation

    for epoch_i in range(epochs):
        print(f'======== Epoch {epoch_i + 1} / {epochs} ========')
        total_train_loss = 0
        
        model.train()
        for step, batch in enumerate(train_dataloader):
            b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]  # Assuming batch is already a tuple
            
            model.zero_grad()
            outputs = model(b_input_ids, attention_mask=b_input_mask)  # model should only return the logits now
            # Assuming labels are already properly shaped (batch_size, 32, 70)
            loss = criterion(outputs.view(-1, 3), b_labels.view(-1))  # Reshape for CrossEntropyLoss
            
            loss = loss / gradient_accumulation_steps
            loss.backward()
            
            total_train_loss += loss.item()
            if (step + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                model.zero_grad()
            
            if step % 40 == 0 and not step == 0:
                print(f'  Batch {step:>5,} of {len(train_dataloader):>5,}. Loss: {loss.item():.2f}')
        
        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"  Average training loss: {avg_train_loss:.2f}")

        print("\nRunning Validation...")
        total_eval_loss = 0
        model.eval()

        for batch in validation_dataloader:
            b_input_ids, b_input_mask, b_labels = [t.to(device) for t in batch]
            
            with torch.no_grad():
                outputs = model(b_input_ids, attention_mask=b_input_mask)
                loss = criterion(outputs.view(-1, 3), b_labels.view(-1))
                total_eval_loss += loss.item()
        
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        print(f"  Validation Loss: {avg_val_loss:.2f}")

    print("Training complete.")


In [51]:
num_labels = len(set(train_sentences_segment))
# num_labels = len(unique_labels)
# num_labels=5
lstm_hidden_dim = 64
bert_model = 'dbmdz/bert-base-turkish-cased'
epochs = 5
batch_size = 16
gradient_accumulation_steps = 5

# model = BertBiLSTMCRF(bert_model, num_labels, lstm_hidden_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()

#model = BertBiLSTMCRF(bert_model, num_labels, lstm_hidden_dim)
#model = BertCRF(bert_model, num_labels)
model = DiacritizationModel(bert_model, hidden_dim=256)

#model.bert = model.bert.to(device)
# Do some operations if necessary
#model.lstm = model.lstm.to(device)
#model.classifier = model.classifier.to(device)
#model.crf = model.crf.to(device)

model.to(device)


train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=True)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False, pin_memory=True)

In [52]:
train_model(model, train_loader, validation_loader, device, epochs, gradient_accumulation_steps)

======== Epoch 1 / 5 ========


/usr/local/src/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
/usr/local/src/pytorch/aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
